# Profiling your code

The Mila Research Template leverages built-in PyTorch and Lightning functionality to make model profiling and benchmarking accessible and flexible.  
Make sure to read the Mila Docs page on [PLACEHOLDER - profiling](https://docs.mila.quebec/) before going through this example. 

The research template profiling notebook extends the examples in the official documentation with additional tools: notably, native WandB integration to monitor performance and using hydra multiruns to compare the available GPUs on the official Mila cluster. See below. The goal of this notebook is to introduce profiling, present tools useful for doing so and to provide general concepts and guidelines for optimizing your code, within the Mila cluster ecosystem.


### Setup

In [1]:
import os
from pathlib import Path
# Set the working directory to the project root
notebook_path = Path().resolve()  
project_root = notebook_path.parent.parent
os.chdir(str(project_root))

## Introduction

As a deep learning researcher, training comparatively slow models as opposed to faster, optimized ones can greatly impact your research output. In addition, as a user of a shared cluster, being efficient about the use of institutional resources is a benefit to all the users in the ecosystem. Given the ample variety of available resources and training schemes to achieve the same modeling objective, optimizing your code isn't necessarily a straightforward task. 

While there's many costs involved in getting a model to train, some are more relevant than others when it comes to making your code more efficient. Setting a performance baseline, by observing said costs and identifying underperforming components in the code while properly contextualizing them within a broader training scheme is the very first step to optimizing your code. Once a baseline performance expectation is set, we can modify and observe our code's performance in a comparative manner to then determine if the performed optimizations are better.

## Instrumenting your code

Setting up artifacts within your code to monitor metrics of interest can help set a cost baseline and evidence potential areas for improvement. Common metrics to watch for include but are not limited to:
 
- Training speed (samples/s)
- CPU/GPU utilization 
- RAM/VRAM utilization

In the Mila ResearchTemplate, this can be done by passing a callback to the trainer. Supported configs are found within the project template at `configs/trainer/callbacks`. Here, we will use the default callback, which in turn implements early stopping and tracks the learning rate, device utilisation and throughput, each through a specific callback instance.

In [14]:
!python project/main.py \
    algorithm=no_op \
    datamodule=imagenet \
    trainer=profiling \
    trainer/callbacks=default

Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
[09/11/24 14:43:09] INFO     Unable to properly create the    ]8;id=107891;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py\auto_schema.py]8;;\:]8;id=976209;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py#368\368]8;;\
                             schema for                                         
                             experiment/cluster_sweep_example                   
                             .yaml last time. Trying again.                     
Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
[09/11/24 14:43:10] WARNING  Unable to create a schema for    ]8;id=289962;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/

### Optional: log metrics on wandb

In addition to callback specification, the Mila Research template integrates wandb as a logger specification, which has enables the tracking of additional metrics through visualizations and dashboard creation.

In [15]:
!python project/main.py \
    algorithm=no_op \
    datamodule=imagenet \
    trainer=profiling \
    trainer/logger=wandb

Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
[09/11/24 14:45:32] INFO     Unable to properly create the    ]8;id=182872;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py\auto_schema.py]8;;\:]8;id=964435;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py#368\368]8;;\
                             schema for                                         
                             experiment/cluster_sweep_example                   
                             .yaml last time. Trying again.                     
Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
                    WARNING  Unable to create a schema for    ]8;id=108986;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py\auto_schema.py]8;;\:]8;id=875791;file:///home/mila/c/ces

We can now visualize the results of our run at `wandb_url`

## Identifying potential bottleneck sources 

Finding a bottleneck is not necessarily clear or straightforward from the start. A sensible first step is to determine whether a potential slowdown originates from data loading or model computation. Querying both ends of the process can be done to determine whether the master process has a significant stall when fetching the next batch, or not. 
If it's close to 0, then data loading outpaces compute, and compute is the bottleneck. 
If it's much greater than 0, then compute outpaces data loading, and data loading is the bottleneck. 
You might not care about CPU usage by the master process and data loaders, so long as the GPU remains fully utilized.

In [3]:
!python project/main.py \
    algorithm=no_op \
    datamodule=imagenet \
    trainer=profiling

Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
[09/11/24 12:09:08] INFO     Unable to properly create the    ]8;id=626004;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py\auto_schema.py]8;;\:]8;id=19589;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py#368\368]8;;\
                             schema for                                         
                             experiment/cluster_sweep_example                   
                             .yaml last time. Trying again.                     
Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
Creating schemas for Hydra config files...   0% ━━━━ 0/46 [ 0:0… < -:-… , ?    ]
                    WARNING  Unable to create a schema for    ]8;id=693194;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py\auto_schema.py]8;;\:]8;id=60895;file:///home/mila/c/cesar

In [15]:
!python project/main.py \
    algorithm=example \
    datamodule=imagenet \
    trainer=profiling

Creating schemas for Hydra config files...   0% ━━━━ 0/45 [ 0:0… < -:-… , ?    ]
[09/10/24 16:16:47] INFO     Unable to properly create the    ]8;id=791467;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py\auto_schema.py]8;;\:]8;id=848502;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py#368\368]8;;\
                             schema for                                         
                             experiment/cluster_sweep_example                   
                             .yaml last time. Trying again.                     
Creating schemas for Hydra config files...   0% ━━━━ 0/45 [ 0:0… < -:-… , ?    ]
Creating schemas for Hydra config files...   0% ━━━━ 0/45 [ 0:0… < -:-… , ?    ]
[09/10/24 16:16:48] WARNING  Unable to create a schema for    ]8;id=992568;file:///home/mila/c/cesar.valdez/idt/ResearchTemplate/project/utils/auto_schema.py\auto_schema.py]8;;\:]8;id=88644;file:///home/mila/c/cesa

## GPU vs CPU

Do we even need a GPU? Compare speed using CPU only vs the slowest GPU available, for a low number of steps
If the CPU performance loosely comparable (for instance, only 1.5-2x slower) than with a GPU, then it might be worth considering! (LMK if this happens, one thing could be to try to increase the # of CPUs and measure performance scaling, then ship this kind of job to a DRAC cluster)
In most workflows, using a GPU actually helps a lot.


In [23]:
!HYDRA_FULL_ERROR=1 python project/main.py \
    algorithm=example \
    datamodule=imagenet \
    trainer=profiling \
    resources=cpu

Traceback (most recent call last):
  File "/home/mila/c/cesar.valdez/idt/ResearchTemplate/project/main.py", line 177, in <module>
    main()
  File "/home/mila/c/cesar.valdez/idt/ResearchTemplate/.venv/lib/python3.10/site-packages/hydra/main.py", line 94, in decorated_main
    _run_hydra(
  File "/home/mila/c/cesar.valdez/idt/ResearchTemplate/.venv/lib/python3.10/site-packages/hydra/_internal/utils.py", line 394, in _run_hydra
    _run_app(
  File "/home/mila/c/cesar.valdez/idt/ResearchTemplate/.venv/lib/python3.10/site-packages/hydra/_internal/utils.py", line 457, in _run_app
    run_and_report(
  File "/home/mila/c/cesar.valdez/idt/ResearchTemplate/.venv/lib/python3.10/site-packages/hydra/_internal/utils.py", line 223, in run_and_report
    raise ex
  File "/home/mila/c/cesar.valdez/idt/ResearchTemplate/.venv/lib/python3.10/site-packages/hydra/_internal/utils.py", line 220, in run_and_report
    return func()
  File "/home/mila/c/cesar.valdez/idt/ResearchTemplate/.venv/lib/python3.10

In [ ]:
!HYDRA_FULL_ERROR=1 python project/main.py \
    algorithm=example \
    datamodule=imagenet \
    trainer=profiling \
    resources=one_gpu

## Testing for throughput across GPUs

As the Mila Research template is built with hydra as a configuration manager, it integrates [Multi-runs](https://hydra.cc/docs/tutorials/basic/running_your_app/multi-run/) by default. This makes it possible to sweep over different parameters for profiling or throughput testing purposes or both. For example, suppose we wanted to figure out how different GPUs perform relative to each other.  
[Mila's official documentation](https://docs.mila.quebec/Information.html) has a comprehensive rundown of the GPUs that are installed on the cluster. Typing ```savail``` on the command line, when logged into the cluster, shows their current availability. Testing their capacity can yield insights into their suitability for different training cases.

In [1]:
!savail

GPU               Avail / Total 
2g.20gb              12 / 48 
3g.40gb               0 / 48 
4g.40gb               0 / 24 
a100                  0 / 32 
a100l                 0 / 88 
a6000                 0 / 8 
rtx8000               8 / 408 
v100                  2 / 56 


As these jobs are part of the cluster, [Submitit](https://hydra.cc/docs/plugins/submitit_launcher/)

We can observe the following prominent GPU classes: a100, a100l, a6000, rtx8000, v100 and MiG partitions with sizes 2g.20gb, 3g.40gb, 4g.40gb.  
We will now proceed to specify different GPUs over training runs and compare their throughput.

In [ ]:
## What performance do you get with each type of GPU? 
# (Based on the VRAM requirements of the job (step 1), 
# try all the GPU types on the Cluster that can accommodate this kind of job)

# Add an example of a sweep over some parameters, 
# with the training throughput as the metric, 
# :: callbacks/samples_per_second, 
# or add a devicestatsmonitor in
# and using different kinds of GPUs. 

## salloc --gres=gpu:a100:1 -c 6 --mem=32G -t 48:00:00 --partition=unkillable

Making sense of the former: if a GPU with lower maximum capacity is readily available, training on it may be more time and resource effective than waiting for higher capacity GPUs to become available.


## GPU utilization

How well are we using the GPU?
Once we've selected the target GPU that we want to use, measure the GPU utilization. Is the GPU utilization high? (>80%?)
If it's high (>80%), then we can either stop here, or we can keep going a bit further
If it's low, then we can use the PyTorch profiler (or any other tool) to try to figure out what the bottleneck i
## maybe look at submitit's array_parallelism

We would like to maximize our throughput given GPU choice

In [ ]:
### Measure the performance on different GPUS using the optimal datamodule 
### params from before (and keeping other parameters the same)

We will now sweep over model hyper-parameters to maximize the utilization of our selected GPU.

In [ ]:
#### Using the results from before, do a simple sweep over model hyper-parameters 
#### to maximize the utilization of the selected GPU (which was selected as a tradeoff 
#### between performance and difficulty to get an allocation). For example if the 
#### RTX8000's are 20% slower than A100s but 5x easier to get an allocation on, use those instead.

## What is a profiler and what is it good for?

The former process was a bit contrived - We can zero down specifically on subprocesses...  
A profiler is a tool that allows you to measure the time and memory consumption of the model’s operators. Specifically, the PyTorch profiler output provides clues about operations relevant to model training. Examples include the total amount of time spent doing low-level mathematical operations in the GPU, and whether these are unexpectedly slow or take a disproportionate amount of time, indicating they should be avoided or optimized. Identifying problematic operations can greatly help us validate or rethink our baseline model performance expectations.

[Multiple](https://developer.nvidia.com/blog/profiling-and-optimizing-deep-neural-networks-with-dlprof-and-pyprof/) [profilers](https://github.com/plasma-umass/scalene) [exist](https://docs.python.org/3/library/profile.html). For the purposes of this example we'll use the default [PyTorch Profiler](https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html). 

In [18]:
from torch.profiler import ProfilerActivity, profile 

profiler = profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=True,
    profile_memory=True,
    with_stack=True,
)
profiler.start()
profiler.stop()
print(profiler.key_averages().table(sort_by="cpu_time_total", row_limit=10))


-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
    cudaDeviceSynchronize       100.00%      14.444us       100.00%      14.444us      14.444us           0 b           0 b             1  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 14.444us



### Additional resources

[GPU Training (Basic) - LightningAI](https://lightning.ai/docs/pytorch/stable/accelerators/gpu_basic.html)  
[DeviceStatsMonitor class - LightningAI](https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.callbacks.DeviceStatsMonitor.html)  
[PyTorch Profiler + W&B integration - Weights & Biases](https://wandb.ai/wandb/trace/reports/Using-the-PyTorch-Profiler-with-W-B--Vmlldzo5MDE3NjU)   
[Advanced profiling for model optimization - Accelerating Generative AI with PyTorch: Segment Anything, Fast](https://pytorch.org/blog/accelerating-generative-ai/)